In [1]:
import time
import tqdm
import csv
import tqdm

In [2]:
csv.field_size_limit(100000000)
csvpath = "data/all_sampled2.csv"
num_articles = 1e10

news = []

with open(csvpath) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    t0 = time.time()
    for i, row in enumerate(csv_reader):
        # if i%1000==0:
        #   print(i)
        if i>=num_articles:
            break
        article = ",".join(row[7:-3])
        date = row[2].strip().split(' ')[0].replace('-', '')[1:7]+"01"
        source = row[-1].split("'")[1]
        link = row[-3]
        
        news_source_dict = {}
        news_source_dict['date'] = date
        news_source_dict['news_source'] = source
        news_source_dict['url'] = link
        news_source_dict['article'] = article
        
        news.append(news_source_dict)
    


    t1 = time.time() - t0
    print(t1)


13.287055969238281


In [53]:
preds = open("data/preds.txt").read().split('\n')[:-1]
preds_ = list(map(lambda x: x.split(' ')[1], preds))
# print(preds_)

for i, elem in enumerate(news):
    if i>=len(preds_):
        break
    elem['pred_label'] = preds_[i]

news = news[:len(preds_)]

news_dict = {}
for elem in news:
#     key = elem['news_source']
    key = elem['news_source']
    if key in news_dict:
        news_dict[key].append(elem)
    else:
        news_dict[key] = [elem]
        
for key in news_dict:
    print(len(news_dict[key]))

76061
21913
14364
2996
16165
2050
10348
18714
4961
9558
8547
4034
3044
1334
2869
4991
4456
5225
4626
2046
9072
438
1614
1207
951
1


In [4]:
count=0
news_dict2 = {}
for key in news_dict:
    num = len(news_dict[key])
    count+=num
    if num>=9000:
        news_dict2[key] = news_dict[key]


print(len(news_dict2))

count=0
for key in news_dict2:
    count+=len(news_dict2[key])
    num = len(news_dict2[key])
    print(key, num)

print(count)

news_dict2["Reuters"][0]

8
Reuters 76061
The New York Times 21913
People 14364
CNBC 16165
Vice 10348
The Hill 18714
CNN 9558
Mashable 9072
176195


{'date': '20170801',
 'news_source': 'Reuters',
 'url': " 'https://www.reuters.com/article/brief-ve-wong-to-pay-2016-dividend-on-oc/brief-ve-wong-to-pay-2016-dividend-on-oct-5-idUSL4N1KX33L'",
 'article': " 'BRIEF-VE WONG to pay 2016 dividend on Oct. 5', 'Aug 11 (Reuters) - VE WONG CORP : * Says it will pay cash dividend of T$1 per share for 2016 to shareholders on Oct. 5 Source text in Chinese: goo.gl/438Y2D Further company coverage: (Beijing Headline News)'",
 'pred_label': 'TECH'}

In [34]:
news_dict3 = {}
# for key in news_dict2:
#     news_dict3[key] = {}
    
for key in news_dict2:
#     years = [elem['date'][:4] for elem in news_dict2[key]]
    row = {}
    for elem in news_dict2[key]:
        year = elem['date'][:1]
#         year = elem['date']

        if year in row:
            row[year].append(elem)
        else:
            row[year] = [elem]
        
    news_dict3[key] = row
#     print(len(row['2019']))


count = 0
for key in news_dict3:
    print(key)
    for key2 in news_dict3[key]:
        print(key2, len(news_dict3[key][key2]))

# print(news_dict3['Reuters']['2016'][100])


Reuters
2 76061
The New York Times
2 21913
People
2 14364
CNBC
2 16165
Vice
2 10348
The Hill
2 18714
CNN
2 9558
Mashable
2 9072


In [46]:
news_dict4 = {}
for key in news_dict3:
    news_dict4[key] = {}
    for key2 in news_dict3[key]:
        row = {}
        for elem in news_dict3[key][key2]:
            label = elem["pred_label"]
            if label in row:
                row[label].append(elem)
            else:
                row[label] = [elem]
        news_dict4[key][key2] = row

category_list = []
count = 0
for key in news_dict4:
    for key2 in news_dict4[key]:
        for key3 in news_dict4[key][key2]:
            elem = news_dict4[key][key2]
            num = len(news_dict4[key][key2][key3])
#             print(key3, num)
            category_list.append(key3)
            count+=num
        
#         break
#     break
category_list = sorted(list(set(category_list)))

In [47]:

# df = []
# for key in news_dict4:
#     for key2 in news_dict4[key]:
#         res = []
#         for key3 in category_list:
#             if key3 in news_dict4[key][key2]:
#                 num = len(news_dict4[key][key2][key3])
#                 num = str(num)
#             else:
#                 num = ''
#             res.append(num)

#         df.append([key]+[key2]+res)
# #             write_res = csv_template.format(key, key2, *res)
# #             outfile.write(write_res)



In [56]:

df = []
for key in news_dict4:
    tot = len(news_dict[key])
    for key2 in news_dict4[key]:
        res = []
        for key3 in category_list:
            if key3 in news_dict4[key][key2]:
                num = len(news_dict4[key][key2][key3])
                frac = 100.*num/tot
                num = str(num)
            else:
                num = ''
                frac = 0
            res.append(str(frac))

        df.append([key]+[key2]+res)
#             write_res = csv_template.format(key, key2, *res)
#             outfile.write(write_res)




In [48]:
# CHOSEN_CATS = ["CRIME", "ENTERTAINMENT", "POLITICS", "SPORTS", "TECH"]
# df = []
# for key in news_dict4:
#     for key2 in news_dict4[key]:
#         res = []
#         tot = 0
#         for key3 in category_list:
#             if key3 in news_dict4[key][key2]:
#                 num = len(news_dict4[key][key2][key3])
#                 if key3 in CHOSEN_CATS:
#                     tot+=num
#         for key3 in category_list:
#             if key3 in news_dict4[key][key2]:
#                 num = len(news_dict4[key][key2][key3])
#                 frac = 1.*num/tot
#                 num = str(num)
#             else:
#                 frac = 0
#                 num = ''
#             res.append(str(frac))
        

#         df.append([key]+[key2]+res)
# #             write_res = csv_template.format(key, key2, *res)
# #             outfile.write(write_res)


In [57]:
df_sort = df.copy()
df_sort.sort(key = lambda x: x[1])
df_sort = [["Publisher", "date"]+ category_list] + df_sort

import pandas as pd

# df_T = pd.DataFrame(df_sort).T.values.tolist()
# df_T.pop(1)

csv_template = ""
for _ in range(len(category_list)):
    csv_template+='{},'
csv_template+='{}\n'

with open("data/barreal2.csv", 'w') as outfile:
    for row in df_sort:
        row.pop(1)
        outrow = csv_template.format(*row)
        outfile.write(outrow)

In [14]:
a = [11,22,33,44]
a.pop(2)
a

[11, 22, 44]